In [1]:
#upload package
import pandas as pd
import numpy as np
import time
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import SVMpkg
# Seed for reproducibility
np.random.seed(0)

In [2]:
#upload dataset
train_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/train.csv", header = None, names = ['variance','skewness','curtosis','entropy','y'])
test_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/test.csv", names = ['variance','skewness','curtosis','entropy','y'])

features = ['variance','skewness','curtosis','entropy']
outcome = 'y'

X_train = train_data[features].values #change to matrix multiple
y_train = train_data[outcome].values
X_test = test_data[features].values
y_test = test_data[outcome].values
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [3]:
# PrimalSVM
Cs = [100 / 873, 500 / 873, 700 / 873]
gamma_values = [1, 0.5, 0.1, 0.01, 0.001]
a_values = [1, 10, 50, 100]
N = 1
epochs = 100

# Initialize variables to track the best parameters and lowest error for each schedule and C
best_params_per_schedule = {"schedule1": {}, "schedule2": {}}

# Perform grid search
for schedule in ["schedule1", "schedule2"]:
    for C in Cs:
        lowest_error_for_C = float("inf")
        best_params_for_C = None
        for gamma in gamma_values:
            for a in a_values:
                # Initialize and train the SVM
                
                svm = SVMpkg.PrimalSVM(gamma=gamma, a=a, C=C, N=N)
                svm.fit(X_train, y_train, epochs=epochs, schedule=schedule)

                # Calculate training and test errors
                train_error = svm.score(X_train, y_train)
                test_error = svm.score(X_test, y_test)
                weights = svm.get_weights()
                bias = svm.get_bias()

                #print(f"Schedule: {schedule}, C={C:.6f}, gamma={gamma}, a={a}, "
                #f"train_error: {train_error:.4f}, test_error: {test_error:.4f},"
                #f"weights, {weights}, bias = {bias}")

                #output objective curve 
                #objective_curve = svm.get_objective_curve()
                #plt.figure(figsize=(8, 5))
                #plt.plot(range(1, len(objective_curve) + 1), objective_curve, marker='o')
                #plt.title("Objective Function Curve (Hinge Loss)")
                #plt.xlabel("Epoch")
                #plt.ylabel("Hinge Loss")
                #plt.grid(True)
                #plt.show()

                # Update the best parameters for the current C
                if test_error < lowest_error_for_C:
                    lowest_error_for_C = test_error
                    best_params_for_C = {
                        "C": C,
                        "gamma": gamma,
                        "a": a,
                        "schedule": schedule,
                        "train_error": train_error,
                        "test_error": test_error,
                        "weights": weights,
                        "bias": bias
                    }
        
        # Store the best parameters for the current C and schedule
        best_params_per_schedule[schedule][C] = best_params_for_C

# Print the best parameters for each C, separated by schedule
for schedule, params_per_C in best_params_per_schedule.items():
    print(f"\nBest Parameters for {schedule}:")
    for C, params in params_per_C.items():
        print(f"C={C:.6f}, Schedule: {params['schedule']}, gamma={params['gamma']}, a={params['a']}, "
              f"train_error: {params['train_error']:.4f}, test_error: {params['test_error']:.4f}, "
              f"weights: {params['weights']}, bias: {params['bias']}")



Best Parameters for schedule1:
C=0.114548, Schedule: schedule1, gamma=0.5, a=100, train_error: 0.0608, test_error: 0.0840, weights: [-0.15217176 -0.11147137 -0.05579284 -0.01562807], bias: 0.14298246924389865
C=0.572738, Schedule: schedule1, gamma=0.01, a=1, train_error: 0.0241, test_error: 0.0180, weights: [-0.32441602 -0.17212906 -0.18608215 -0.02517079], bias: 0.5004602638064877
C=0.801833, Schedule: schedule1, gamma=0.001, a=100, train_error: 0.0183, test_error: 0.0160, weights: [-0.34762413 -0.18979735 -0.22631406 -0.00247417], bias: 0.6468729605176408

Best Parameters for schedule2:
C=0.114548, Schedule: schedule2, gamma=1, a=50, train_error: 0.0631, test_error: 0.0860, weights: [-0.15368874 -0.11439085 -0.05609331 -0.01529727], bias: 0.1352745796472408
C=0.572738, Schedule: schedule2, gamma=1, a=10, train_error: 0.0218, test_error: 0.0160, weights: [-0.32397074 -0.17102877 -0.18428299 -0.02530887], bias: 0.4902974613772339
C=0.801833, Schedule: schedule2, gamma=1, a=10, train_e

In [4]:
#DualSVM
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873

# Train and evaluate the model for different values of C
for C in Cs:
    svm = SVMpkg.DualSVM(C)
    svm.fit(X_train, y_train)
    train_error = svm.score(X_train, y_train)
    test_error = svm.score(X_test, y_test)
    print(f"C={C}, Train error={train_error:.2f}, Test error={test_error:.2f}")
    print(f"Weights:, {svm.w}, Bias:, {svm.b}")

# Print the weights and bias for the best model
print("Weights:", svm.w)
print("Bias:", svm.b)

d:\EID-Download\02 Academic\Python38\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


C=0.1145475372279496, Train error=0.11, Test error=0.12
Weights:, [-0.94292598 -0.65149184 -0.73372197 -0.04102195], Bias:, 4.14119177534919
C=0.572737686139748, Train error=0.15, Test error=0.14
Weights:, [-1.56393784 -1.01405165 -1.18065044 -0.15651687], Bias:, 7.590350666124916
C=0.8018327605956472, Train error=0.40, Test error=0.43
Weights:, [-2.04254833 -1.28068891 -1.51351532 -0.24905307], Bias:, 12.975949611428163
Weights: [-2.04254833 -1.28068891 -1.51351532 -0.24905307]
Bias: 12.975949611428163


In [ ]:
#GaussianSVM
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873
gammas = [0.1, 0.5, 1, 5, 100]

lowest_error = float("inf")
best_params = None

for C in Cs:
    for gamma in gammas:
        gsvm = SVMpkg.GaussianSVM(C, gamma)
        gsvm.fit(X_train, y_train)
        sv_indices = gsvm.get_support_vectors()
        train_error = gsvm.score(X_train, y_train)
        test_error = gsvm.score(X_test, y_test)
        print(f"C={C:.4f}, gamma={gamma}, Train error={train_error:.4f}, Test error={test_error:.4f}, Number of Support Vectors: {len(sv_indices)}")

        if test_error < lowest_error:
            lowest_error = test_error
            best_params = (C, gamma)

print(f"Best Parameters: C={best_params[0]:.4f}, gamma={best_params[1]}")
print(f"Best Test Error: {lowest_error:.4f}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -82.72279493439663
            Iterations: 15
            Function evaluations: 13098
            Gradient evaluations: 15
C=0.1145, gamma=0.1, Train error=0.4461, Test error=0.4420, Number of Support Vectors: 853
Optimization terminated successfully    (Exit mode 0)
            Current function value: -74.16551934430456
            Iterations: 23
            Function evaluations: 20084
            Gradient evaluations: 23
C=0.1145, gamma=0.5, Train error=0.4071, Test error=0.4260, Number of Support Vectors: 745


In [ ]:
#GaussianSVM
C = 500 / 873 
gammas = [0.01, 0.1, 0.5, 1, 5]

# Track support vectors
support_vectors = {}
for gamma in gammas:
    gsvm = SVMpkg.GaussianSVM(C=C, gamma=gamma)
    gsvm.fit(X_train, y_train)
    sv_indices = gsvm.get_support_vectors()
    support_vectors[gamma] = sv_indices
    print(f"Gamma={gamma}, Number of Support Vectors: {len(sv_indices)}")

# Calculate overlaps between consecutive gammas
for i in range(len(gammas) - 1):
    gamma1, gamma2 = gammas[i], gammas[i + 1]
    overlap = len(np.intersect1d(support_vectors[gamma1], support_vectors[gamma2]))
    print(f"Overlap between gamma={gamma1} and gamma={gamma2}: {overlap}")

In [ ]:
#Kernel Perceptron
gammas = [0.1, 0.5, 1, 5, 100]
for gamma in gammas:
    kp = SVMpkg.KernelPerceptron(gamma=gamma)
    kp.fit(X_train, y_train)
    train_error = kp.score(X_train, y_train)
    test_error = kp.score(X_test, y_test)
    print(f"Gamma={gamma}, Train error ={train_error:.4f}, Test error ={test_error:.4f}")
